In [ ]:
import os
import json

import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date
import requests
from ast import literal_eval

from bqplot import Lines, Figure, LinearScale, DateScale, Axis, Hist

from ipyleaflet import Map, GeoJSON, WidgetControl, ImageOverlay
from ipywidgets import Layout, Dropdown

In [ ]:

# res = requests.get("https://raw.githubusercontent.com/RichardQZeng/Notebooks/main/footprint_fixed.geojson")
# gdf = gpd.read_file(res.text)
gdf = gpd.read_file(r"C:\BERATools\Notebooks\footprint_fixed-4326.geojson")

In [ ]:
def array_eval(x):
    array_str = ','.join(x.split())
    array_str = array_str.replace('[,', '[')
    array_str = array_str.replace('\n', '')
    return np.array(literal_eval(array_str))
    
gdf['sampling_widths'] = gdf['sampling_widths'].apply(array_eval)

In [ ]:
gdf['OLnFID']

In [ ]:
def clean_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data


def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.0)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range


def extrap_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data[column] = data[column].apply(extrap_interp)
    return data

In [ ]:
line_id = 0
x_data = gdf[gdf['OLnFID'] == 0]['sampling_widths'].values[0]

In [ ]:
x_data

In [ ]:
x_sc = LinearScale()
y_sc = LinearScale()

hist = Hist(sample=x_data, scales={"sample": x_sc, "count": y_sc})
ax_x = Axis(scale=x_sc, tick_format="0.2f")
ax_y = Axis(scale=y_sc, orientation="vertical")
hist.bins = 10

figure = Figure(marks=[hist], axes=[ax_x, ax_y], padding_y=0)

In [ ]:
def update_figure(line_id):
    hist.sample = gdf[gdf['OLnFID'] == line_id]['sampling_widths'].values[0]
    ax_y.label = str(line_id)

In [ ]:
update_figure(line_id)

In [ ]:
with open("./countries.geo.json") as f:
    countries = json.load(f)

In [ ]:
with open(r"C:\BERATools\Notebooks\footprint_fixed-4326.geojson") as f:
    lines = json.load(f)

with open(r"C:\BERATools\Notebooks\footprint-4326.geojson") as f:
    lines_org = json.load(f)

In [ ]:
m = Map(center=(56.20112069, -110.67053897),
        zoom=14,
        scroll_wheel_zoom=True,
        layout=Layout(height='700px'))

In [ ]:
geo = GeoJSON(
    data=lines,
    style={"fillColor": "white", "weight": 0.5, 'opacity': 0.5,},
    name="Line footprint",
)
m.add(geo)

geo_org = GeoJSON(
    data=lines_org,
    style={"fillColor": "rgb(54, 49, 92)", "weight": 1},
    hover_style={"fillColor": "#148ee6"},
    name="Line footprint",
)
m.add(geo_org)

widget_control1 = WidgetControl(widget=figure, position="bottomright")
widget_control1.min_width = 400

m.add(widget_control1)


def on_hover(event, feature, **kwargs):
    global line_id

    line_id = feature["properties"]["OLnFID"]
    update_figure(line_id)
    print("feature", feature)


geo.on_hover(on_hover)

In [ ]:
data_name = "income"
dropdown = Dropdown(
    options=["income", "population", "lifeExpectancy"],
    value=data_name,
    description="Plotting:",
)


def on_click(change):
    global footrpint_type

    footrpint_type = change["new"]
    print(change["new"])
    # update_figure(footrpint_type, line_id)


dropdown.observe(on_click, "value")

widget_control2 = WidgetControl(widget=dropdown, position="bottomleft")

# m.add(widget_control2)

In [ ]:
m